In [80]:
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.python.keras.utils import np_utils
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectFromModel, f_classif, mutual_info_classif, SelectKBest, VarianceThreshold, SelectPercentile


#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_dropfinal_withoutSMILES_appendclass_dropunmane.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#X 정규화

scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=10)

#SVM 모델

svm = SVC(C = 30, gamma  = 1e-05, kernel = 'linear')
svm.fit(X_train, Y_train)

pred_train = svm.predict(X_train)
pred_test = svm.predict(X_test)
pred_accuracy = accuracy_score(Y_test, pred_test)

print('SVM 정확도: {0:.4f}'.format(pred_accuracy))

SVM 정확도: 0.4286


In [13]:
#SVM 파리미터 튜닝(SVC)

import pandas as pd
import tensorflow as tf
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.python.keras.utils import np_utils
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectFromModel, f_classif, mutual_info_classif, SelectKBest, VarianceThreshold, SelectPercentile
from scipy.stats import uniform, randint
from sklearn.utils.fixes import loguniform


#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_Internal_dataset_dropunmane.csv')
df2 = pd.read_csv('./VRK2_inhibitor_descriptor_external_dataset_dropunmane.csv')
df3 = pd.concat([df, df2])

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

dataset2 = df2.values
exX = dataset[:,0:-1]
exY = dataset[:,-1]

dataset3 = df3.values
totX = dataset[:,0:-1]
totY = dataset[:,-1]

#X 정규화

scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)
exX_scaled = scaler.transform(exX)
totX_scaled = scaler.transform(totX)

#학습셋과 테스트셋을 나눔

#X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=10)

#SVM 모델튜닝
#param_grid = {'C': np.arange(0.000015, 0.000005, 0.000001), 'gamma': np.arange(0.0000001, 0.000001, 0.0000001),
#              'kernel': ['linear','rbf','poly','sigmoid']}
#svm = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
#svm.fit(X_train, Y_train)

#print(svm.best_params_)

#pred_train = svm.predict(X_train)
#pred_test = svm.predict(X_test)
#pred_accuracy = accuracy_score(Y_test, pred_test)

#파이프라인 생성
sel = SelectKBest()
clf = SVC()
step = [('sel',sel), ('svc', clf) ]
pipeline = Pipeline(step)
parameter =[{'sel__k':randint(10, 3330), 'svc__C': ran}]

print(pipeline.get_params().keys())
 
grid = RandomizedSearchCV(pipeline, parameter, n_iter=5, cv=5, scoring = 'accuracy', n_jobs = -1)
grid.fit(X_scaled, Y)

print("Best Parameters : ", grid.best_params_)
print("Best Score : ", grid.best_score_)

grid_clf = grid.best_estimator_
grid_clf.fit(X_scaled, Y)

print('테스트 정확도: %.3f'% grid_clf.score(exX_scaled, exY))
print('total 정확도: %.3f'% grid_clf.score(totX_scaled, totY))

# Save best grid search pipeline to file
saved_model_svc = joblib.dump(grid_clf,'./VRK2_svc5.pkl')
#{'C': 1e-05, 'gamma': 1e-07, 'kernel': 'poly'} SVM 정확도: 0.7143 

#피처별 중요도 알아봐야함

dict_keys(['memory', 'steps', 'verbose', 'sel', 'svc', 'sel__k', 'sel__score_func', 'svc__C', 'svc__break_ties', 'svc__cache_size', 'svc__class_weight', 'svc__coef0', 'svc__decision_function_shape', 'svc__degree', 'svc__gamma', 'svc__kernel', 'svc__max_iter', 'svc__probability', 'svc__random_state', 'svc__shrinking', 'svc__tol', 'svc__verbose'])
Best Parameters :  {'sel__k': 866}
Best Score :  0.5133333333333334
테스트 정확도: 0.731
total 정확도: 0.731


C:\Users\eunwoo\.conda\envs\Tensorflow\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\eunwoo\.conda\envs\Tensorflow\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  41  363 1223 1507 1510 2355 2374 2680 2685 2690 2694 2705 2722 2752
 2757 2761 2783 2806 2828 2849 2860 2885 2895 2903 2919 2921 2923 2928
 2929 2930 2931 2932 2937 2941 2946 2951 2956 2957 2961 2962 2967 2970
 2972 2973 2976 2983 2990 2998 3000 3007 3009 3018 3019 3020 3021 3027
 3036 3046 3047 3051 3057 3060 3062 3066 3073 3080 3088 3090 3097 3099
 3108 3110 3170 3171 3185 3186 3187 3188 3195 3196 3198 3202 3214 3215
 3216 3217 3218 3219 3221 3235 3236 3237 3238 3248 3250 3251 3252 3256
 3260 3261 3262 3277 3278 3279 3280 3282 3283 3284 3296 3297 3298 3299
 3300 3302 3310 3311 3312 3313 3314 3315 331

In [191]:
#SVM 파리미터 튜닝(LinearSVC)

import pandas as pd
import tensorflow as tf
import joblib


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.utils import np_utils
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectFromModel, f_classif, mutual_info_classif, SelectKBest, VarianceThreshold, SelectPercentile
from scipy.stats import uniform, randint

#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_dropfinal_withoutSMILES_appendclass_dropunmane.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#X 정규화

scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=10)

#파이프라인 생성

sel = SelectKBest() #K는 130 혹은 119
clf = LinearSVC()
step = [('sel',sel), ('clf', clf) ]
pipeline = Pipeline(step)
parameter = {'sel__k':randint(10, 3330), 'clf__penalty': ['l2'], 'clf__max_iter': [18], 'clf__loss':['squared_hinge'],
            'clf__dual':[True], 'clf__C': [0.8], 'clf__multi_class':['ovr'], 'clf__fit_intercept': [True],
            'clf__intercept_scaling': [0.72362], 'clf__random_state': [0]}
grid = RandomizedSearchCV(pipeline, parameter, n_iter=100000, cv=5, scoring = 'accuracy', n_jobs = -1)
grid.fit(X_train, Y_train)
saved_model_svc = joblib.dump(grid,'./VRK2_linearsvc2.pkl')

print("Best Parameters : ", grid.best_params_)
print("Best Score : ", grid.best_score_)
print("Best Test Score : ", grid.score(X_test, Y_test))


Best Parameters :  {'clf__C': 0.8, 'clf__dual': True, 'clf__fit_intercept': True, 'clf__intercept_scaling': 0.72362, 'clf__loss': 'squared_hinge', 'clf__max_iter': 18, 'clf__multi_class': 'ovr', 'clf__penalty': 'l2', 'clf__random_state': 0, 'sel__k': 119}
Best Score :  0.64
Best Test Score :  0.7142857142857143


C:\Users\eunwoo\.conda\envs\Tensorflow\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\eunwoo\.conda\envs\Tensorflow\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  41  363 1203 2689 2690 2693 2694 2704 2705 2752 2753 2756 2757 2761
 2783 2806 2828 2849 2860 2885 2903 2916 2921 2923 2929 2930 2932 2941
 2949 2951 2957 2962 2970 2973 2976 2983 2990 2992 2998 3000 3007 3009
 3019 3020 3039 3047 3060 3066 3073 3080 3082 3088 3090 3097 3099 3110
 3170 3171 3186 3187 3188 3195 3196 3198 3202 3216 3217 3218 3219 3221
 3235 3236 3237 3238 3247 3248 3250 3251 3252 3256 3260 3261 3262 3278
 3279 3280 3282 3283 3284 3295 3296 3297 3298 3299 3300 3302 3310 3311
 3312 3313 3314 3315 3316 3334 3335 3336] are constant.
  warnings.warn("Features %s are constant." % constant_feat

In [207]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, f_classif, mutual_info_classif, SelectKBest, VarianceThreshold, SelectPercentile

#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_dropfinal_withoutSMILES_appendclass_dropunmane.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#X 정규화

scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=10)


Polynomial_svm_clf = Pipeline([('sel', VarianceThreshold()), ('poly_features', PolynomialFeatures()), 
                               ('svm_clf',LinearSVC())])
parameter = {'sel__'}
grid = GridSearchCV(Polynomial_svm_clf, param_grid = parameter, cv=5, scoring = 'accuracy', n_jobs = -1)
grid.fit(X_train,Y_train)
pred = grid.predict(X)
pred_accuracy = accuracy_score(Y, pred)

print('SVM 정확도: {0:.4f}'.format(pred_accuracy))

SVM 정확도: 0.1562


In [20]:
#SVM 파리미터 튜닝(LinearSVC)

import pandas as pd
import tensorflow as tf
import joblib


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.python.keras.utils import np_utils
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectFromModel, f_classif, mutual_info_classif, SelectKBest, VarianceThreshold, SelectPercentile
from scipy.stats import uniform, randint

#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_dropfinal_withoutSMILES_appendclass_dropunmane.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#X 정규화

scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=10)

#파이프라인 생성

sel = SelectKBest() #K는 130 혹은 119
clf = LinearSVC()
step = [('sel',sel), ('clf', clf) ]
pipeline = Pipeline(step)
parameter = {'sel__k':randint(10, 3330), 'clf__penalty': ['l1','l2'], 'clf__max_iter': randint(100, 5000), 
             'clf__loss':['squared_hinge', 'hinge'],
            'clf__dual':[True, False], 'clf__C': uniform(1, 1e10),
            'clf__intercept_scaling': uniform(0.001, 1), 'clf__random_state': [10]}
grid = RandomizedSearchCV(pipeline, parameter, n_iter=10000, cv=5, scoring = 'accuracy', n_jobs = -1)
grid.fit(X_train, Y_train)
saved_model_svc = joblib.dump(grid,'./VRK2_linearsvc6.pkl')

print("Best Parameters : ", grid.best_params_)
print("Best Score : ", grid.best_score_)
print("Best Test Score : ", grid.score(X_test, Y_test))

C:\Users\eunwoo\.conda\envs\Tensorflow\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\eunwoo\.conda\envs\Tensorflow\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  41  363 1203 2689 2690 2693 2694 2704 2705 2752 2753 2756 2757 2761
 2783 2806 2828 2849 2860 2885 2903 2916 2921 2923 2929 2930 2932 2941
 2949 2951 2957 2962 2970 2973 2976 2983 2990 2992 2998 3000 3007 3009
 3019 3020 3039 3047 3060 3066 3073 3080 3082 3088 3090 3097 3099 3110
 3170 3171 3186 3187 3188 3195 3196 3198 3202 3216 3217 3218 3219 3221
 3235 3236 3237 3238 3247 3248 3250 3251 3252 3256 3260 3261 3262 3278
 3279 3280 3282 3283 3284 3295 3296 3297 3298 3299 3300 3302 3310 3311
 3312 3313 3314 3315 3316 3334 3335 3336] are constant.
  warnings.warn("Features %s are constant." % constant_feat

Best Parameters :  {'clf__C': 3205160915.7198925, 'clf__dual': True, 'clf__intercept_scaling': 0.8515436913506029, 'clf__loss': 'hinge', 'clf__max_iter': 4967, 'clf__penalty': 'l2', 'clf__random_state': 10, 'sel__k': 124}
Best Score :  0.64
Best Test Score :  1.0
